In [ ]:
from asapdiscovery.data.operators.state_expanders.protomer_expander import (
    EpikExpander,
    ProtomerExpander,
)
from asapdiscovery.docking.docking import DockingResult
from asapdiscovery.docking.scorer import ChemGauss4Scorer
from asapdiscovery.data.schema.complex import PreppedComplex, Complex
from asapdiscovery.data.schema.ligand import Ligand
from asapdiscovery.docking.docking import DockingInputPair
from asapdiscovery.data.testing.test_resources import fetch_test_file

In [ ]:
# Load du
prepped_complex = PreppedComplex.from_oedu_file(
        fetch_test_file("Mpro-P2660_0A_bound-prepped_receptor.oedu"),
        ligand_kwargs={"compound_name": "test"},
        target_kwargs={"target_name": "test", "target_hash": "mock_hash"},
    )

In [ ]:
expanded_ligands = ProtomerExpander().expand(ligands=[Ligand.from_smiles(prepped_complex.ligand.smiles, **prepped_complex.ligand.dict())])

In [ ]:
expanded_ligands

In [ ]:
from openeye import oequacpac, oechem
lig = prepped_complex.ligand
oemol = lig.to_oemol()
for protomer in oequacpac.OEGetReasonableProtomers(oemol):
    fmol = oechem.OEMol(protomer)
    # copy the ligand properties over to the new molecule, we may want to have more fine grained control over this
    # down the track.
    protomer_ligand = Ligand.from_oemol(fmol, **lig.dict())

In [ ]:
lig = prepped_complex.ligand
lig.data

In [ ]:
oemol = lig.to_oemol()

In [ ]:
ligs = []
for mol in oequacpac.OEEnumerateFormalCharges(oemol):
    ligs.append(Ligand.from_oemol(mol, **lig.dict()))

In [ ]:
print([lig.smiles for lig in ligs])

In [ ]:
for i,lig in enumerate(ligs):
    if i == 0:
        lig.compound_name = "deprotonated"
    else:
        lig.compound_name = "protonated"

In [ ]:
ligs.reverse()

In [ ]:
docking_input = DockingInputPair(complex=prepped_complex, ligand=prepped_complex.ligand)

In [ ]:
docking_result = DockingResult(input_pair=docking_input,posed_ligand=prepped_complex.ligand,provenance={"test": "test"})

In [ ]:
docking_results = []
for lig in ligs:
    docking_input = DockingInputPair(complex=prepped_complex, ligand=lig)
    docking_result = DockingResult(input_pair=docking_input,posed_ligand=lig,provenance=lig.provenance)
    docking_results.append(docking_result)

In [ ]:
docking_results

In [ ]:
docking_result.write_docking_files(".")

In [ ]:
chemgauss_scorer = ChemGauss4Scorer()

In [ ]:
scores = chemgauss_scorer.score(docking_results, return_df=True)

In [ ]:
scores

# Use constrained pose generation

In [ ]:
from asapdiscovery.docking.schema.pose_generation import OpenEyeConstrainedPoseGenerator

In [ ]:
pose_gen = OpenEyeConstrainedPoseGenerator()

In [ ]:
poses = pose_gen.generate_poses(prepped_complex, ligs)

In [ ]:
for lig in poses.posed_ligands:
    lig.to_sdf(f"{lig.smiles}.sdf")

In [ ]:
docking_results = []
for lig in poses.posed_ligands:
    docking_input = DockingInputPair(complex=prepped_complex, ligand=lig)
    docking_result = DockingResult(input_pair=docking_input,posed_ligand=lig,provenance=lig.provenance)
    docking_results.append(docking_result)

In [ ]:
chemgauss_scorer = ChemGauss4Scorer()
scores = chemgauss_scorer.score(docking_results, return_df=True)

In [ ]:
scores

## the difference in score means that the change in protonation state of the ligand does affect the score

strangely, the protonated ligand has a lower score than the deprotonated ligand

In [ ]:
for result in docking_results:
    result.write_docking_files(f"{result.input_pair.ligand.compound_name}")

In [ ]:
protonated_complex = Complex.from_pdb("protonated/test-mock_hash+JZJCSVMJFIAMQB-DLYUOGNHNA-N_protonated-JZJCSVMJFIAMQB-YZUISDQHNA-O/docked_complex.pdb", target_kwargs=dict(target_name="test"), ligand_kwargs=dict(compound_name="protonated"))

In [ ]:
deprotonated_complex = Complex.from_pdb("deprotonated/test-mock_hash+JZJCSVMJFIAMQB-DLYUOGNHNA-N_deprotonated-JZJCSVMJFIAMQB-DLYUOGNHNA-N/docked_complex.pdb", target_kwargs=dict(target_name="test"), ligand_kwargs=dict(compound_name="deprotonated"))

In [ ]:
scorer = ChemGauss4Scorer()

In [ ]:
scorer.score([protonated_complex, deprotonated_complex])

In [ ]:
prepped_complexes = [PreppedComplex.from_complex(complex) for complex in [protonated_complex, deprotonated_complex]]

In [ ]:
scorer = ChemGauss4Scorer()
scorer.score(prepped_complexes)

In [ ]:
from openeye import oedocking

In [ ]:
scores = []
for complex in prepped_complexes:
    posed_mol = complex.ligand.to_oemol()
    pose_scorer = oedocking.OEScore(oedocking.OEScoreType_Chemgauss4)
    du = complex.target.to_oedu()
    pose_scorer.Initialize(du)
    chemgauss_score = pose_scorer.ScoreLigand(posed_mol)
    scores.append((complex.ligand.compound_name, chemgauss_score))

In [ ]:
print([c.ligand.smiles for c in prepped_complexes])

In [ ]:
for c in prepped_complexes:
    c.ligand.to_sdf(f"{c.ligand.compound_name}.sdf")

# this doesn't work because after prepping the complexes, the sdf files are the same

In [ ]:
scores = []
for prepped_complex in prepped_complexes:
    pose_scorer = oedocking.OEScore(oedocking.OEScoreType_Chemgauss4)
    du = prepped_complex.target.to_oedu()
    pose_scorer.Initialize(du)
    for complex in [protonated_complex, deprotonated_complex]:
        posed_mol = complex.ligand.to_oemol()
        chemgauss_score = pose_scorer.ScoreLigand(posed_mol)
        scores.append((f"Prepped Complex: {prepped_complex.ligand.compound_name}", complex.ligand.compound_name, chemgauss_score))

In [ ]:
scores

In [ ]:
from asapdiscovery.data.backend.openeye import save_openeye_sdfs

In [ ]:
save_openeye_sdfs([c.ligand.to_oemol() for c in [protonated_complex, deprotonated_complex]], "multiligand.sdf")

In [ ]:
from openeye import oeshape
# Prepare reference molecule for calculation
# With default options this will remove any explicit
# hydrogens present, and add required color atoms
refmol = protonated_complex.ligand.to_oemol()
fitmol = deprotonated_complex.ligand.to_oemol()
prep = oeshape.OEOverlapPrep()
prep.Prep(refmol)

# Get appropriate function to calculate exact color
func = oeshape.OEOverlapFunc()
func.SetupRef(refmol)

res = oeshape.OEOverlapResults()
prep.Prep(fitmol)
func.Overlap(fitmol, res)
print("title: %s  color score = %.2f" %
      (fitmol.GetTitle(), res.GetColorTanimoto()))

In [ ]:
res.GetColorScore()